In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ライブラリのインポート
import pandas as pd # 基本ライブラリ
import numpy as np # 基本ライブラリ
import matplotlib.pyplot as plt # グラフ描画用
import seaborn as sns; sns.set() # グラフ描画用
import warnings # 実行に関係ない警告を無視
warnings.filterwarnings('ignore')
import lightgbm as lgb #LightGBM
from sklearn import datasets
from sklearn.model_selection import train_test_split # データセット分割用
from sklearn.metrics import mean_squared_error # モデル評価用(平均二乗誤差)
from sklearn.metrics import r2_score # モデル評価用(決定係数)

# データフレームを綺麗に出力する関数
import IPython
def display(*dfs, head=True):
    for df in dfs:
        IPython.display.display(df.head() if head else df)

# データの読み込み

In [3]:
# air_reserve
df_ar = pd.read_csv('/kaggle/input/air-base-data/air_reserve.csv')
# air_store_info
df_as = pd.read_csv('/kaggle/input/air-base-data/air_store_info.csv')
# air_visit_data
df_av = pd.read_csv('/kaggle/input/air-base-data/air_visit_data.csv')
# air_store_info_with_nearest_active_station
df_nearest_active_station = pd.read_csv('/kaggle/input/rrv-weather-data/air_store_info_with_nearest_active_station.csv')



In [4]:
# preped-tokyo-tokyokana-tonokyo
df_weather_info = pd.read_csv('../input/weather-info-target-station/tokyo__tokyo-kana__tonokyo.csv')


In [ ]:
# データの確認
print(df_ar.shape) # データサイズの確認(データ数,特徴量数(変数の個数))
display(df_ar) # df.head()に同じ(文中に入れるときはdisplay()を使う)

In [93]:
print(df_as.shape) # データサイズの確認(データ数,特徴量数(変数の個数))
display(df_as) # df.head()に同じ(文中に入れるときはdisplay()を使う)

In [92]:
print(df_av.shape) # データサイズの確認(データ数,特徴量数(変数の個数))
display(df_av) # df.head()に同じ(文中に入れるときはdisplay()を使う)

In [ ]:
print(df_nearest_active_station.shape) # データサイズの確認(データ数,特徴量数(変数の個数))
display(df_nearest_active_station) # df.head()に同じ(文中に入れるときはdisplay()を使う)

In [76]:
print(df_weather_info.shape) # データサイズの確認(データ数,特徴量数(変数の個数))
display(df_weather_info) # df.head()に同じ(文中に入れるときはdisplay()を使う)

# データ準備

In [5]:
df_nearest_active_station_as = pd.merge(df_nearest_active_station, df_as, on='air_store_id',how='inner')
display(df_nearest_active_station_as) 


In [6]:
df_taget_store = df_nearest_active_station_as.query('station_id.str.contains("tonoky")', engine='python')
display(df_taget_store) 


In [7]:
# 本分析では、station「tokyo__tokyo-kana__tonokyo」が最寄りのストアを対象に、分析する 
df_target_av_as = pd.merge(df_av, df_taget_store, on='air_store_id',how='inner')
display(df_target_av_as) 

In [8]:
#天気情報を追加
df_target_av_as.rename(columns={'visit_date': 'target_date'}, inplace=True)
df_weather_info.rename(columns={'calendar_date': 'target_date'}, inplace=True)

df_target_av_as_wf = pd.merge(df_target_av_as, df_weather_info, on='target_date',how='inner')
display(df_target_av_as_wf) 


# 前処理(特徴量エンジニアリング)

In [9]:
# =============================================================================
# 日付
# =============================================================================
time_format = '%Y-%m-%d'
def seperate_date(data):     
    # split date feature in real visit datetime
    data_time = pd.to_datetime(data.target_date, format=time_format)
    data['Year_visit']= data_time.dt.year
    data['Month_visit'] = data_time.dt.month
    data['DayOfYear_visit'] = data_time.dt.dayofyear
    # data['DayOfMonth_visit'] = data_time.dt.day
#    data['WeekOfYear_visit'] = data_time.dt.week
    data['DayOfWeek_visit'] = data_time.dt.dayofweek
#    del data['visit_date']
    return data


In [10]:
seperate_date(df_target_av_as_wf)
display(df_target_av_as_wf)

In [11]:
# =============================================================================
# label encoding
# =============================================================================
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
lb = preprocessing.LabelBinarizer()

le.fit(df_target_av_as_wf['air_store_id'])
df_target_av_as_wf['air_store_id_label'] = le.transform(df_target_av_as_wf['air_store_id'])

le.fit(df_target_av_as_wf['air_genre_name_x'])
df_target_av_as_wf['air_genre_name_label'] = le.transform(df_target_av_as_wf['air_genre_name_x'])

display(df_target_av_as_wf)

In [12]:
dataset = df_target_av_as_wf.drop(
#    ["air_area_name_x","latitude_x","longitude_x","latitude_str","longitude_str","station_id","station_latitude","station_longitude","station_vincenty","station_great_circle","air_genre_name_y","air_area_name_y","latitude_y","longitude_y"],axis=1)
    ["air_area_name_x","latitude_x","longitude_x","latitude_str","longitude_str","station_id","station_latitude","station_longitude","station_vincenty","station_great_circle","air_genre_name_y","air_area_name_y","latitude_y","longitude_y","air_store_id","target_date","air_genre_name_x","deepest_snowfall","total_snowfall"],axis=1)
display(dataset)

In [13]:
#トレーニングデータ・検証用・テストデータの分割
# 説明変数,目的変数
X = dataset.drop('visitors',axis=1).values # 説明変数
y = dataset['visitors'].values # 目的変数(来店数)


In [14]:
feature_names = dataset.drop('visitors',axis=1).columns.tolist()
print(feature_names)

In [166]:
print(X)

In [167]:
print(y)

In [15]:
#デフォルトはtest_size=0.25で25%がテスト用、残りの75%が訓練用となる。
#デフォルトでは要素がシャッフルされて分割される。
#シャッフルされる場合、デフォルトでは実行するたびにランダムに分割される。引数random_stateを指定して乱数シードを固定すると常に同じように分割される。
#機械学習のモデルの性能を比較するような場合、どのように分割されるかによって結果が異なってしまうため、乱数シードを固定して常に同じように分割されるようにする必要がある。

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=2)
X_train_, X_valid, y_train_, y_valid = train_test_split(X_train, y_train,test_size=0.10, random_state=2)


# モデルの構築

In [16]:
# モデルの学習
d_train = lgb.Dataset(data=X_train_, label=y_train_, categorical_feature=['air_store_id_label', 'air_genre_name_label'],feature_name = feature_names)
d_valid = lgb.Dataset(data=X_valid,  label=y_valid,categorical_feature=['air_store_id_label', 'air_genre_name_label'],feature_name = feature_names)


watchlist = [d_train, d_valid]

print('Training LGBM model...')
params = {}
params['application'] = 'regression'
params['boosting'] = 'gbdt' # 勾配ブースティング
params['learning_rate'] = 0.01 # 学習率
params['num_leaves'] = 32 # 葉の数
params['min_sum_hessian_in_leaf'] = 1e-2
params['min_gain_to_split'] = 0

params['bagging_fraction'] = 0.8
params['feature_fraction'] = 0.8 # 学習の高速化と過学習の抑制に使用される．データの特徴量のfeature_fraction * 100 % だけ使用する．
params['num_threads'] = 4
params['metric'] = 'rmse'



In [17]:
lgb_model1 = lgb.train(params, train_set=d_train, num_boost_round=10000, valid_sets=watchlist, \
verbose_eval=10)

# モデルの評価

In [18]:
lgb.plot_importance(lgb_model1, figsize=(9,20)) #plot_importance...変数重要度

In [19]:
#参考
#https://qiita.com/sudachi0114/items/ebabef57bc19d5624682
from sklearn.metrics import mean_squared_error
y_test_pred = lgb_model1.predict(X_test)
print(y_test_pred)

rmse = mean_squared_error(y_test,y_test_pred)

print(rmse)
